# **ENSEMBLE resnext50_32x4d and efficient net b3 with Albumentations Transform**

In [ ]:
#!pip install ../input/pytorch-image-models/timm-0.3.1-py3-none-any.whl ../input/walkwithfastai/wwf-0.0.5-py3-none-any.whl

In [ ]:
from fastai.vision.all import *

In [ ]:
class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)
    
    
    


In [ ]:

def get_x(row): return row['image_id']
def get_y(row): return row['label']

In [ ]:
#from wwf.vision.timm import *

In [ ]:
#!pip install wwf timm 


In [ ]:
learn=load_learner("../input/resnext50/baseline_rsnx",cpu=False)

In [ ]:
learn = learn.to_native_fp32()

In [ ]:
data_path="../input/cassava-leaf-disease-classification/"

In [ ]:
sample_df = pd.read_csv(data_path+'sample_submission.csv')
sample_df.head()

In [ ]:
sample_copy = sample_df.copy()
sample_copy['image_id'] = sample_copy['image_id'].apply(lambda x: "../input/cassava-leaf-disease-classification/test_images/"+x)

In [ ]:
test_dl = learn.dls.test_dl(sample_copy)

In [ ]:
test_dl.one_batch()[0].shape

In [ ]:
preds, _ = learn.tta(dl=test_dl, n=15, beta=0)

In [ ]:
#learn=load_learner("../input/efficient-net/baseline_eff",cpu=False)

In [ ]:
#learn = learn.to_native_fp32()

In [ ]:
#test_dl = learn.dls.test_dl(sample_copy)

In [ ]:
#preds2, _ = learn.tta(dl=test_dl, n=15, beta=0)

In [ ]:
#predx=(preds+preds2)/2

In [ ]:
sample_df['label'] = preds.argmax(dim=-1).numpy()

In [ ]:
sample_df.to_csv('submission.csv',index=False)

In [ ]:
pd.read_csv("./submission.csv")